<a href="https://colab.research.google.com/github/Venkatesh-lol/DL-Transfer-Learning/blob/main/experiment_Transfer_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
tf.random.set_seed(42)
np.random.seed(42)

LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
          tf.keras.layers.Dense(300, kernel_initializer = "he_normal"),
          tf.keras.layers.LeakyReLU(),
          tf.keras.layers.Dense(100, kernel_initializer= "he_normal"),
          tf.keras.layers.LeakyReLU(),
          tf.keras.layers.Dense(10, activation="softmax")]

model = tf.keras.models.Sequential(LAYERS)

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data = (X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 5s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/10
1719/1719 - 4s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/10
1719/1719 - 4s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/10
1719/1719 - 4s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/10
1719/1719 - 4s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006
Epoch 6/10
1719/1719 - 4s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052
Epoch 7/10
1719/1719 - 4s - loss: 0.3644 - accuracy: 0.8980 - val_loss: 0.3348 - val_accuracy: 0.9102
Epoch 8/10
1719/1719 - 4s - loss: 0.3485 - accuracy: 0.9021 - val_loss: 0.3209 - val_accuracy: 0.9138
Epoch 9/10
1719/1719 - 4s - loss: 0.3356 - accuracy: 0.9053 - val_loss: 0.3111 - val_accuracy: 0.9152
Epoch 10/10
1719/1719 - 4s - loss: 0.3251 - accuracy: 0.9077 - val_loss: 0.3016 - 

In [8]:
model.save("pretrained_mnist_model.h5")

# Transfer learning

In [9]:
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")

In [10]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [11]:
for  layer in pretrained_mnist_model.layers:
  print(f"{layer.name}:{layer.trainable}")

flatten:True
dense:True
leaky_re_lu:True
dense_1:True
leaky_re_lu_1:True
dense_2:True


In [14]:
for layer in pretrained_mnist_model.layers[:-1]:
  layer.trainable = False
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False


In [15]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1]
new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(
    tf.keras.layers.Dense(, activation="sigmoid")
)

In [16]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 265,701
Trainable params: 101
Non-trainable params: 265,600
______________________________________________

In [18]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [19]:
np.where(y_train%2 ==0, 1,0)

array([0, 0, 1, ..., 0, 1, 1])

In [20]:
y_train[0],y_train[-1]

(7, 8)

In [41]:
def update_even_odd_labels(labels):
  for idx, label in enumerate(labels):
    labels[idx]=np.where(label > 5, 1,0)
  return labels

In [42]:
y_train_bin, y_test_bin, y_valid_bin = update_even_odd_labels([y_train, y_test, y_valid])

In [43]:
np.unique(y_valid_bin)

array([0, 1])

In [44]:
new_model.compile(loss="binary_crossentropy",
                  optimizer=tf.keras.optimizers.SGD(lr=1e-3),
                  metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [45]:
history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data = (X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 3s - loss: 0.6217 - accuracy: 0.6949 - val_loss: 0.4418 - val_accuracy: 0.7840
Epoch 2/10
1719/1719 - 3s - loss: 0.4090 - accuracy: 0.8156 - val_loss: 0.3809 - val_accuracy: 0.8378
Epoch 3/10
1719/1719 - 3s - loss: 0.3738 - accuracy: 0.8478 - val_loss: 0.3549 - val_accuracy: 0.8616
Epoch 4/10
1719/1719 - 3s - loss: 0.3567 - accuracy: 0.8608 - val_loss: 0.3416 - val_accuracy: 0.8726
Epoch 5/10
1719/1719 - 3s - loss: 0.3467 - accuracy: 0.8663 - val_loss: 0.3320 - val_accuracy: 0.8778
Epoch 6/10
1719/1719 - 3s - loss: 0.3400 - accuracy: 0.8695 - val_loss: 0.3258 - val_accuracy: 0.8794
Epoch 7/10
1719/1719 - 3s - loss: 0.3351 - accuracy: 0.8712 - val_loss: 0.3206 - val_accuracy: 0.8800
Epoch 8/10
1719/1719 - 3s - loss: 0.3311 - accuracy: 0.8731 - val_loss: 0.3166 - val_accuracy: 0.8800
Epoch 9/10
1719/1719 - 3s - loss: 0.3279 - accuracy: 0.8741 - val_loss: 0.3133 - val_accuracy: 0.8818
Epoch 10/10
1719/1719 - 3s - loss: 0.3251 - accuracy: 0.8752 - val_loss: 0.3101 - 

In [46]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.3169 - accuracy: 0.8766


[0.3168613612651825, 0.8766000270843506]

In [47]:
X_new = X_test[:3]

y_test[:3], y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([1, 0, 0]))

In [48]:
np.round(new_model.predict(X_new))

array([[1.],
       [0.],
       [0.]], dtype=float32)